In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set_style("whitegrid")

In [87]:
# https://www.kaggle.com/datasets/aslanahmedov/market-basket-analysis/data
data = pd.read_csv('/home/laurinbrechter/Documents/Code/PatternMining/data/kaggle/Assignment-1_Data.csv', delimiter=';')
data['Price'] = data.Price.str.replace(',', '.').astype(float)

/tmp/ipykernel_30526/3535922104.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/home/laurinbrechter/Documents/Code/PatternMining/data/kaggle/Assignment-1_Data.csv', delimiter=';')


In [88]:
data['Value'] = data.Quantity * data.Price
data = data.loc[data.Value > 0]

In [ ]:
data.groupby('Itemname').Price.std().sort_values(ascending=False).head(10)

In [42]:
summary = data.groupby('BillNo').agg(
    avg_price=('Price', 'mean'),
    num_items=('Itemname', 'count'),
    transaction_value=('Value', 'sum')
)

summary_filtered = summary.loc[(summary.transaction_value > 0) & (summary.transaction_value < 5000)]

In [43]:
summary.num_items.mean()

26.56055416389755

In [44]:
summary.transaction_value.mean()

526.6548220438627

In [45]:
data.BillNo.nunique()


19561

In [46]:
data.Itemname.nunique()

4006

In [47]:
value_by_item = data.groupby('Itemname').Value.sum().sort_values(ascending=False)

In [ ]:
cum_item_sum = (value_by_item / value_by_item.sum()).cumsum()
ax = sns.lineplot(x=range(len(cum_item_sum)), y=cum_item_sum)
# lines at 70% and 95%
sns.lineplot(x=range(len(cum_item_sum)), y=[0.7]*len(cum_item_sum), color='red', linestyle='--')
sns.lineplot(x=range(len(cum_item_sum)), y=[0.95]*len(cum_item_sum), color='red', linestyle='--')

ax.set(xlabel='Items', ylabel='Cumulative Value')

In [ ]:
sns.histplot(summary_filtered.transaction_value, bins=100)

In [137]:
data['item_id'] = pd.factorize(data.Itemname)[0].astype(str)
data["Value_Int"] = data["Value"].astype(int).astype(str)
data = data.loc[data.Value_Int != '0']

In [140]:
transaction_db = data.groupby('BillNo').agg(
    items=('item_id', lambda x: ' '.join(list(x))),
    total_value=('Value', lambda x: int(x.sum())),
    values=('Value_Int', lambda x: ' '.join(list(x))),
    num_items=('item_id', 'count'),
    # num_values=('Value_Int', 'count')
)
transaction_db = transaction_db.loc[transaction_db.num_items < 10].iloc[:1000]

In [141]:
transaction_db

,items,total_value,values,num_items
BillNo,,,,
536365,0 1 2 3 4 5 6,139,15 20 22 20 20 15 25,7
536366,7 8,22,11 11,2
536368,21 22 23 24,70,25 14 14 14,4
536369,25,17,17,1
536371,46,204,204,1
...,...,...,...,...
543016,135,165,165,1
543017,1055,15,15,1
543019,516 608,21,12 8,2


In [114]:
transaction_db.to_csv('tdb.csv', header=False, sep=':', index=False)

In [115]:
from PAMI.extras.dbStats.utilityDatabaseStats import utilityDatabaseStats 
obj = utilityDatabaseStats('tdb.csv', sep=' ')
obj.run()
obj.printStats()

Database size : 1000
Number of items : 1274
Minimum Transaction Size : 1
Average Transaction Size : 4.186
Maximum Transaction Size : 9
Minimum utility : 1
Average utility : 295.1773940345369
Maximum utility : 77183
Standard Deviation Transaction Size : 2.7432469812249862
Variance : 7.532936936936937
Sparsity : 0.9967142857142857


In [116]:
import PAMI.highUtilityPattern.basic.EFIM as efim 
import PAMI.highUtilityFrequentPattern.basic.HUFIM as hufim

In [117]:
# obj = hufim.HUFIM('tdb.csv', minUtil=500, sep=' ', minSup=
import PAMI.highUtilityPattern.basic.EFIM as efim 

obj = efim.EFIM('tdb.csv', minUtil=1000, sep=' ') 
obj.startMine()                       #start the mining process 
obj.save('out.txt')               #store the patterns in file 
results = obj.getPatternsAsDataFrame()     #Get the patterns discovered into a dataframe 
obj.printResults()      

High Utility patterns were generated successfully using EFIM algorithm
Total number of High Utility Patterns: 1136
Total Memory in USS: 1120870400
Total Memory in RSS 1138040832
Total ExecutionTime in seconds: 0.12198805809020996


In [118]:
results.sort_values(by='Utility', ascending=False).head(10)

,Patterns,Utility
932,1956 910 1344 1481 196,9931
936,1956 910 1344 1481 129,9931
1058,910 1344 1481 196 129,9906
700,1077 1956 910 1481 120,9896
635,1077 1956 1431 910 1481 129,9883
631,1077 1956 1431 910 1481 196,9883
987,1956 196 129 120,9877
824,1077 910 1481 129 120,9871
822,1077 910 1481 196 120,9871
758,1077 1431 910 1481 196 129,9858


In [133]:
itemnames = pd.factorize(data.Itemname)[1]
results['PatternNames'] = results.Patterns.str.split(' ').apply(lambda x: ' + '.join([itemnames[int(i)] for i in x]))

In [143]:
pd.set_option('display.max_colwidth', None)
results.sort_values(by='Utility', ascending=False).head(10)
# display full column

,Patterns,Utility,PatternNames
932,1956 910 1344 1481 196,9931,VINTAGE BEAD PINK PURSE + ANT COPPER RED BOUDICCA BRACELET + BLUE STRIPE CERAMIC DRAWER KNOB + CLASSICAL ROSE CANDLESTAND + ALARM CLOCK BAKELIKE IVORY
936,1956 910 1344 1481 129,9931,VINTAGE BEAD PINK PURSE + ANT COPPER RED BOUDICCA BRACELET + BLUE STRIPE CERAMIC DRAWER KNOB + CLASSICAL ROSE CANDLESTAND + ENAMEL BREAD BIN CREAM
1058,910 1344 1481 196 129,9906,ANT COPPER RED BOUDICCA BRACELET + BLUE STRIPE CERAMIC DRAWER KNOB + CLASSICAL ROSE CANDLESTAND + ALARM CLOCK BAKELIKE IVORY + ENAMEL BREAD BIN CREAM
700,1077 1956 910 1481 120,9896,MIRRORED WALL ART SKULLS + VINTAGE BEAD PINK PURSE + ANT COPPER RED BOUDICCA BRACELET + CLASSICAL ROSE CANDLESTAND + HANGING MEDINA LANTERN SMALL
635,1077 1956 1431 910 1481 129,9883,MIRRORED WALL ART SKULLS + VINTAGE BEAD PINK PURSE + SKULLS GREETING CARD + ANT COPPER RED BOUDICCA BRACELET + CLASSICAL ROSE CANDLESTAND + ENAMEL BREAD BIN CREAM
631,1077 1956 1431 910 1481 196,9883,MIRRORED WALL ART SKULLS + VINTAGE BEAD PINK PURSE + SKULLS GREETING CARD + ANT COPPER RED BOUDICCA BRACELET + CLASSICAL ROSE CANDLESTAND + ALARM CLOCK BAKELIKE IVORY
987,1956 196 129 120,9877,VINTAGE BEAD PINK PURSE + ALARM CLOCK BAKELIKE IVORY + ENAMEL BREAD BIN CREAM + HANGING MEDINA LANTERN SMALL
824,1077 910 1481 129 120,9871,MIRRORED WALL ART SKULLS + ANT COPPER RED BOUDICCA BRACELET + CLASSICAL ROSE CANDLESTAND + ENAMEL BREAD BIN CREAM + HANGING MEDINA LANTERN SMALL
822,1077 910 1481 196 120,9871,MIRRORED WALL ART SKULLS + ANT COPPER RED BOUDICCA BRACELET + CLASSICAL ROSE CANDLESTAND + ALARM CLOCK BAKELIKE IVORY + HANGING MEDINA LANTERN SMALL
758,1077 1431 910 1481 196 129,9858,MIRRORED WALL ART SKULLS + SKULLS GREETING CARD + ANT COPPER RED BOUDICCA BRACELET + CLASSICAL ROSE CANDLESTAND + ALARM CLOCK BAKELIKE IVORY + ENAMEL BREAD BIN CREAM
